# IMDB 분류

## 합성곱 신경망(CNN)을 활용한 IMDB 분석
---
인공신경망 CNN 주요 모형을 활용한 영화 리뷰 분류

### Dataset 불러오기

#### 학습용 데이터 불러오기 및 정규화

In [11]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
vocab_size = 10000

(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)

In [13]:
X_train[:3]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [15]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [17]:
print('X_train의 크기(shape) :', X_train.shape)
print('X_test의 크기(shape) :', X_test.shape)

X_train의 크기(shape) : (25000, 200)
X_test의 크기(shape) : (25000, 200)


In [18]:
print(y_train[:5])

[1 0 0 1 0]


#### 검증용 데이터 불러오기

### 1D CNN으로 IMDB 리뷰 분류하기

In [19]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [20]:
embedding_dim = 256
dropout_ratio = 0.3
num_filters = 256
kernel_size = 3
hidden_units = 128

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

In [23]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 0.3992 - acc: 0.8067
Epoch 1: val_acc improved from -inf to 0.86488, saving model to best_model.h5
782/782 [==============================] - 106s 135ms/step - loss: 0.3992 - acc: 0.8067 - val_loss: 0.3100 - val_acc: 0.8649
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 0.1975 - acc: 0.9230
Epoch 2: val_acc improved from 0.86488 to 0.89076, saving model to best_model.h5
782/782 [==============================] - 111s 142ms/step - loss: 0.1975 - acc: 0.9230 - val_loss: 0.2677 - val_acc: 0.8908
Epoch 3/20
782/782 [==============================] - ETA: 0s - loss: 0.0869 - acc: 0.9695
Epoch 3: val_acc did not improve from 0.89076
782/782 [==============================] - 114s 146ms/step - loss: 0.0869 - acc: 0.9695 - val_loss: 0.3218 - val_acc: 0.8872
Epoch 4/20
782/782 [==============================] - ETA: 0s - loss: 0.0379 - acc: 0.9877
Epoch 4: val_acc did not improve from 0.89076
782/782 [=

In [25]:
loaded_model = load_model('best_model.h5')
print("\n테스트 정확도: %.4f" %(loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 13s 17ms/step - loss: 0.2677 - acc: 0.8908

테스트 정확도: 0.8908
